In [ ]:
from ner_clsf import NERClassifier

In [ ]:
from anntools import Collection
from pathlib import Path

In [ ]:
# collection = Collection().load_dir(Path('2021/ref/training'))

In [ ]:
ner_clf = NERClassifier()

In [ ]:
# ner_clf.train(collection)

In [ ]:
# ner_clf.save_model('ner')

In [ ]:
dev_set = Collection().load(Path('2021/eval/develop/scenario1-main/input.txt'))

In [ ]:
ner_clf.load_model('ner')

In [ ]:
#     features = self.get_features(collection)
#     X = self.preprocess_features(features, {'dep': 0, 'pos': 0}, train=False)
#     pred = self.model.predict(X, verbose=1)
#     return self.convert_to_label(pred)

In [ ]:
len(dev_set)

In [ ]:
labels, indices = ner_clf.test_model(dev_set)
indices[:10]

In [ ]:
from anntools import Sentence, Relation, Keyphrase
import spacy
nlp = spacy.load('es_core_news_sm')

In [ ]:
dev_set = Collection().load(Path('2021/eval/develop/scenario1-main/input.txt'))

In [ ]:
def get_label(label, pred_label, multiple, sent, next_id, word):
    if label not in pred_label:
        return False, next_id, multiple
    if pred_label == 'B-' + label:
        if multiple:
            sent.keyphrases.append(Keyphrase(sent, label, next_id, multiple))
            next_id += 1
        multiple = []
    try:
        idx = multiple[-1][-1]
    except IndexError:
        idx = 0
    i = sent.text.index(word.text, idx)
    span = i, i + len(word)
    multiple.append(span)
    return True, next_id, multiple

In [ ]:
def complete_sentences(labels, indices, sentences):
    next_id = 0
    for sent_label, index in zip(labels, indices):
        multiple_concepts = []
        multiple_actions = []
        multiple_predicates = []
        multiple_references = []
        sent = sentences[index]
        tokens = nlp(sent.text)
        for label, word in zip(sent_label, tokens):
            concept, next_id, multiple_concepts = get_label('Concept', label, multiple_concepts, sent, next_id, word)
            if not concept:
                action, next_id, multiple_actions = get_label('Action', label, multiple_actions, sent, next_id, word)
                if not action:
                    reference, next_id, multiple_references = get_label('Reference', label, multiple_references, sent, next_id, word)
                    if not reference:
                        _, next_id, multiple_predicates = get_label('Predicate', label, multiple_predicates, sent, next_id, word)
        if multiple_concepts:
            sent.keyphrases.append(Keyphrase(sent, 'Concept', next_id, multiple_concepts))
            next_id += 1
        if multiple_actions:
            sent.keyphrases.append(Keyphrase(sent, 'Action', next_id, multiple_actions))
            next_id += 1
        if multiple_predicates:
            sent.keyphrases.append(Keyphrase(sent, 'Predicate', next_id, multiple_predicates))
            next_id += 1
        if multiple_references:
            sent.keyphrases.append(Keyphrase(sent, 'Reference', next_id, multiple_references))
            next_id += 1

In [ ]:
complete_sentences(labels, indices, dev_set)

In [ ]:
# labels[0]

In [ ]:
dev_set.sentences